In [ ]:
# !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz

In [ ]:
# !tar xzvf food-101.tar.gz

In [ ]:
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB3, InceptionV3, ResNet101
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten 
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.experimental import CosineDecay
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
from google.colab import files

In [ ]:
files.upload()

Saving dataset100.zip to dataset100.zip


In [ ]:
train_path = '../input/food101/training'
val_path = '../input/food101/validation'

# Data Generator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
img_size = 224
img_shape = (img_size,img_size,3)
batch_size = 32
epoch = 10
dropout_rate = 0.5

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode='categorical'
)
val_generator = train_datagen.flow_from_directory(
    val_path,
    target_size=(img_size,img_size),
    batch_size=batch_size,
    class_mode='categorical'
)

# Base model

## EfficientNet B3  
Tan, Mingxing, and Quoc Le. "Efficientnet: Rethinking model scaling for convolutional neural networks." International Conference on Machine Learning. PMLR, 2019.

In [ ]:
efficientnet = EfficientNetB3(
    include_top=False, weights='imagenet',
    input_shape=img_shape, classes = 101,
    classifier_activation='softmax'
)

inputs = Input(shape=img_shape)
efficientnet = efficientnet(inputs)
pooling = layers.GlobalAveragePooling2D()(efficientnet)
dropout = layers.Dropout(dropout_rate)(pooling)
outputs = Dense(101, activation="softmax")(dropout)
model = Model(inputs=inputs, outputs=outputs)

## InceptionV3  
Szegedy, Christian, et al. "Rethinking the inception architecture for computer vision." Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.

In [ ]:
inception = InceptionV3(
    include_top=False, weights='imagenet',
    input_shape=img_shape, classes = 101,
    classifier_activation='softmax')

inputs = Input(shape=img_shape)
inception = inception(inputs)
pooling = layers.GlobalAveragePooling2D()(inception)
dropout = layers.Dropout(dropout_rate)(pooling)
outputs = Dense(101, activation="softmax")(dropout)
model = Model(inputs=inputs, outputs=outputs)

## ResNet101  
He, Kaiming, et al. "Deep residual learning for image recognition." Proceedings of the IEEE conference on computer vision and pattern recognition. 2016.

In [ ]:
ResNet101 = ResNet101(
    include_top=False, weights=None,
    input_shape=img_shape, classes = 101,
    classifier_activation='softmax'
)

inputs = Input(shape=img_shape)
ResNet101 = ResNet101(inputs)
pooling = layers.GlobalAveragePooling2D()(ResNet101)
dropout = layers.Dropout(dropout_rate)(pooling)
outputs = Dense(101, activation="softmax")(dropout)
model = Model(inputs=inputs, outputs=outputs)

# Train

In [ ]:
decay_steps = int(round(90900/batch_size))*epochs
cosine_decay = CosineDecay(initial_learning_rate=1e-4, decay_steps=decay_steps, alpha=0.3)

callbacks = [ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True)]

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(cosine_decay), metrics=["accuracy"])